In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import libraries 
import tensorflow as tf
import keras
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameter as hp
from keras.layers import Dense,Dropout,Activation,Add,MaxPooling2D,Conv2D,Flatten,BatchNormalization,MaxPool2D
from keras.models import Sequential
from keras import layers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications import VGG19,Xception,VGG16

In [ ]:
#import train data
train_datagen = ImageDataGenerator(rescale=1/255,
                                   shear_range = 0.3,
                                   zoom_range = 0.3,horizontal_flip = True,
                                   vertical_flip =  True , 
                                   rotation_range=60,
                                   brightness_range = (0.4, 1.4))


train_data = train_datagen.flow_from_directory('../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train',
                                                 target_size = (244, 244),
                                                 class_mode='sparse',
                                                 shuffle=True,seed=1)
#import test data

test_datagen = ImageDataGenerator(rescale = 1/255)
test_data = test_datagen.flow_from_directory("../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test",
                                                           batch_size=32,
                                                           target_size=(244,244),
                                                           class_mode='sparse',
                                                           shuffle=True,seed=1)


In [ ]:
#def_front example
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/ok_front/cast_ok_0_1019.jpeg"
new_img = image.load_img(image_path, target_size=(250, 250))
print("ok_font")
plt.imshow(new_img)

In [ ]:
#ok example
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/test/def_front/cast_def_0_108.jpeg"
new_img = image.load_img(image_path, target_size=(250, 250))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
print("def_font")
plt.imshow(new_img)

In [ ]:
# viualize data
fig, ax = plt.subplots()
ax.bar(["ok"],[3394],color= "g",label='OK')
ax.bar(["def"],[4211],color ="b",label ="DEF")
ax.legend()

In [ ]:
classes_count = {"def_front" : 4211,"ok_front" :3394 }
classes_labels = {0:"def_front",1:"ok_front"}

In [ ]:
#build our model 
vgg_model =  VGG16(include_top=True , weights='imagenet')
for models in vgg_model.layers:
  models.trainable= False
vgg_model = keras.Model(inputs=vgg_model.input, outputs=vgg_model.layers[-2].output)
vgg = keras.Sequential()
for layer in vgg_model.layers:
  vgg.add(layer)
vgg.add(Dropout(0.2))
vgg.add(Dense(2, activation='softmax'))
vgg.summary()

In [ ]:
#compile my model
vgg.compile(optimizer=keras.optimizers.Adam(0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
#fit our model
h = vgg.fit(train_data,validation_data=test_data,epochs = 10)

In [ ]:
#evulate model 
vgg.evaluate(test_data)

In [ ]:
#predict test data
y_prediction =  vgg.predict(test_data)
y_prediction =  np.argmax(y_prediction,axis = 1 )

print(y_prediction)

In [ ]:
#y_true and y_pred
y_true = np.array([])
y_pred = np.array([])

i = 0
for data, labels in test_data:
  i += 1
  y = np.argmax(vgg.predict(data), axis=1)
  y_true = np.append(y_true, labels)
  y_pred = np.append(y_pred, y)
  
  if i == test_data.samples // 32 + 1:
    break

In [ ]:
import sklearn
cm = sklearn.metrics.confusion_matrix(y_true, y_pred)
print(cm)

In [ ]:
import pandas as pd
import seaborn 
df_cm = pd.DataFrame(cm, index = [i for i in [0,1]],
                  columns = [i for i in [0,1]])
seaborn .heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='d')
plt.title('confusion matrix')
plt.xlabel('prediction')
plt.ylabel('Actual');

In [ ]:
#test_example 1 
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_512x512/casting_512x512/def_front/cast_def_0_1055.jpeg"
new_img = image.load_img(image_path, target_size=(244, 244))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = vgg.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(classes_labels[prediction[0]])
plt.imshow(new_img)

In [ ]:
#test_example 2 
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train/ok_front/cast_ok_0_1016.jpeg"
new_img = image.load_img(image_path, target_size=(244, 244))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = vgg.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(classes_labels[prediction[0]])
plt.imshow(new_img)

In [ ]:
#test_example 3 
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_data/casting_data/train/ok_front/cast_ok_0_1019.jpeg"
new_img = image.load_img(image_path, target_size=(244, 244))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = vgg.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(classes_labels[prediction[0]])
plt.imshow(new_img)

In [ ]:
#test_example 4
image_path = "../input/real-life-industrial-dataset-of-casting-product/casting_512x512/casting_512x512/def_front/cast_def_0_1188.jpeg"
new_img = image.load_img(image_path, target_size=(244, 244))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
prediction = vgg.predict(img)
prediction = np.argmax(prediction,axis=1)
print(prediction)
print(classes_labels[prediction[0]])
plt.imshow(new_img)

In [ ]:
acc = h.history['accuracy']
val_acc = h.history['val_accuracy']
loss = h.history['loss']
val_loss = h.history['val_loss']
epochs = range(1, len(loss) + 1)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# save model
vgg.save("model_casting.h5") 